In [3]:
import plotly.graph_objects as go
import numpy as np

In [3]:
fig = go.Figure(data=[
    go.Mesh3d(
        # 8 vertices of a cube
        x=[0, 0, 1, 1, 0, 0, 1, 1],
        y=[0, 1, 1, 0, 0, 1, 1, 0],
        z=[0, 0, 0, 0, 1, 1, 1, 1],
        # i, j and k give the vertices of triangles
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
    )
])

fig.show()

In [4]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, n_dontknow = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces

In [5]:
with open("../../data/meshes/bull.off") as f:
    verts, faces = read_off(f)
arr_verts, arr_faces = np.array(verts), np.array(faces)

In [1]:
from plotting_utils import read_off
arr_verts, arr_faces = read_off("../../data/meshes/moomoo_s0.off")

In [4]:
fig = go.Figure(data=[
    go.Mesh3d(
        # 8 vertices of a cube
        x=arr_verts[:,0],
        y=arr_verts[:,1],
        z=arr_verts[:,2],
        # i, j and k give the vertices of triangles
        i = arr_faces[:,0],
        j = arr_faces[:,1],
        k = arr_faces[:,2],
        color='cyan', 
    ),
])
#fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )

fig.show()

In [13]:
import numpy as np

def is_point_inside_mesh(point, vertices, triangles):
    def ray_intersects_triangle(ray_origin, ray_direction, triangle_vertices):
        # Möller–Trumbore intersection algorithm
        v0, v1, v2 = triangle_vertices
        e1 = v1 - v0
        e2 = v2 - v0
        h = np.cross(ray_direction, e2)
        a = np.dot(e1, h)

        if a > -1e-6 and a < 1e-6:
            return False

        f = 1.0 / a
        s = ray_origin - v0
        u = f * np.dot(s, h)

        if u < 0.0 or u > 1.0:
            return False

        q = np.cross(s, e1)
        v = f * np.dot(ray_direction, q)

        if v < 0.0 or u + v > 1.0:
            return False

        t = f * np.dot(e2, q)

        return t > 1e-6

    # Check if the point is inside the mesh volume
    ray_origin = point
    ray_direction = np.array([1, 0, 0])  # Choose a ray direction (e.g., along the positive x-axis)

    inside_count = 0

    for triangle_indices in triangles:
        triangle_vertices = vertices[triangle_indices]
        if ray_intersects_triangle(ray_origin, ray_direction, triangle_vertices):
            inside_count += 1

    # If the number of intersections is odd, the point is inside the mesh volume
    return inside_count % 2 == 1

# Example usage:
vertices = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
triangles = np.array([[0, 1, 2], [0, 2, 3]])

point_inside = is_point_inside_mesh([0, 0, 0], arr_verts, arr_faces)
print(f"Is the point inside the mesh volume? {point_inside}")

Is the point inside the mesh volume? True


In [15]:
import numpy as np

def calculate_mesh_volume(vertices, triangles):
    def signed_volume_of_triangle(v1, v2, v3):
        return np.dot(v1, np.cross(v2, v3)) / 6.0

    volume = 0.0

    for triangle_indices in triangles:
        triangle_vertices = vertices[triangle_indices]
        v1, v2, v3 = triangle_vertices
        volume += signed_volume_of_triangle(v1, v2, v3)

    return abs(volume)

# Example usage:
vertices = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
triangles = np.array([[0, 1, 2], [0, 2, 3]])

mesh_volume = calculate_mesh_volume(arr_verts, arr_faces)
print(f"Volume of the mesh: {mesh_volume}")

Volume of the mesh: 0.23629868815459323


In [30]:
import numpy as np
from scipy.sparse import coo_matrix

def cotLaplacian(X, T):
    nv = X.shape[0]
    nf = T.shape[0]

    # Find original edge lengths and angles
    L1 = np.linalg.norm(X[T[:, 1], :] - X[T[:, 2], :], axis=1)
    L2 = np.linalg.norm(X[T[:, 0], :] - X[T[:, 2], :], axis=1)
    L3 = np.linalg.norm(X[T[:, 0], :] - X[T[:, 1], :], axis=1)
    EL = np.column_stack([L1, L2, L3])

    A1 = (L2**2 + L3**2 - L1**2) / (2 * L2 * L3)
    A2 = (L1**2 + L3**2 - L2**2) / (2 * L1 * L3)
    A3 = (L1**2 + L2**2 - L3**2) / (2 * L1 * L2)
    A = np.column_stack([A1, A2, A3])
    A = np.arccos(A)

    # The Cot Laplacian
    I = np.concatenate([T[:, 0], T[:, 1], T[:, 2]])
    J = np.concatenate([T[:, 1], T[:, 2], T[:, 0]])
    S = 0.5 / np.tan(np.concatenate([A[:, 2], A[:, 0], A[:, 1]]))
    In = np.concatenate([I, J, I, J])
    Jn = np.concatenate([J, I, I, J])
    Sn = np.concatenate([-S, -S, S, S])

    # Compute the areas. Use mixed weights Voronoi areas
    cA = 0.5 / np.tan(A)
    vp1 = [2, 0, 1]
    vp2 = [1, 2, 0]
    At = 1/4 * (EL[:, vp1]**2 * cA[:, vp1] + EL[:, vp2]**2 * cA[:, vp2])

    # Triangle areas
    N = np.cross(X[T[:, 0], :] - X[T[:, 1], :], X[T[:, 0], :] - X[T[:, 2], :])
    Ar = np.linalg.norm(N, axis=1)

    # Use barycentric area when cot is negative
    locs = np.where(cA[:, 0] < 0)[0]
    At[locs, 0] = Ar[locs] / 4
    At[locs, 1] = Ar[locs] / 8
    At[locs, 2] = Ar[locs] / 8

    locs = np.where(cA[:, 1] < 0)[0]
    At[locs, 0] = Ar[locs] / 8
    At[locs, 1] = Ar[locs] / 4
    At[locs, 2] = Ar[locs] / 8

    locs = np.where(cA[:, 2] < 0)[0]
    At[locs, 0] = Ar[locs] / 8
    At[locs, 1] = Ar[locs] / 8
    At[locs, 2] = Ar[locs] / 4

    # Vertex areas = sum triangles nearby
    I = np.concatenate([T[:, 0], T[:, 1], T[:, 2]])
    J = np.ones_like(I)
    S = np.concatenate([At[:, 0], At[:, 1], At[:, 2]])

    W = coo_matrix((Sn, (In, Jn)), shape=(nv, nv)).tocsr()
    #A = coo_matrix((S, (I, J)), shape=(nv, 1)).tocsr()

    return W, A

def normv(V):
    return np.sqrt(np.sum(V**2, axis=1))

In [32]:
W, A = cotLaplacian(arr_verts, arr_faces)

In [33]:
W

<502x502 sparse matrix of type '<class 'numpy.float64'>'
	with 3502 stored elements in Compressed Sparse Row format>

In [1]:
import numpy as np
from scipy.sparse import coo_matrix

def cotangent_laplacian(vertices, triangles):
    """
    Compute the cotangent Laplacian matrix for a closed triangular mesh.

    Parameters:
    - vertices: Numpy array with the coordinates of the vertices.
    - triangles: Numpy array with the triangles' edge indices.

    Returns:
    - Laplacian matrix.
    """
    num_vertices = vertices.shape[0]
    laplacian_data = []
    row_indices = []
    col_indices = []

    # Iterate over each triangle
    for triangle in triangles:
        # Extract vertex indices for the triangle
        i, j, k = triangle

        # Calculate edge vectors
        v1 = vertices[j] - vertices[i]
        v2 = vertices[k] - vertices[i]

        # Calculate cotangent of the angles
        cot_alpha = np.dot(-v1, v2) / np.linalg.norm(np.cross(v1, v2))
        cot_beta = np.dot(v1, -v2) / np.linalg.norm(np.cross(v1, v2))
        cot_gamma = np.dot(v2, -v1) / np.linalg.norm(np.cross(v1, v2))

        # Update Laplacian matrix entries
        laplacian_data.extend([-cot_alpha, -cot_beta, cot_alpha + cot_beta, -cot_alpha, cot_beta, cot_alpha])
        row_indices.extend([i, j, k, i, j, k])
        col_indices.extend([j, k, i, k, i, j])

    # Construct the Laplacian matrix in COO format
    laplacian_matrix = coo_matrix((laplacian_data, (row_indices, col_indices)), shape=(num_vertices, num_vertices))

    # Convert to CSR format for efficient computations
    laplacian_matrix = laplacian_matrix.tocsr()

    # Compute the diagonal matrix D
    diagonal_matrix = np.diag(np.array(laplacian_matrix.sum(axis=1)).flatten())

    # Compute the cotangent Laplacian matrix L = D - Laplacian
    cot_laplacian_matrix = diagonal_matrix - laplacian_matrix

    return cot_laplacian_matrix

# Example usage:
vertices = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0]])
triangles = np.array([[0, 1, 2], [1, 2, 3]])

laplacian_matrix = cotangent_laplacian(arr_verts, arr_faces)
print("Cotangent Laplacian Matrix:")
print(laplacian_matrix.toarray())

NameError: name 'arr_verts' is not defined

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from skimage import io, color

def image_resize(x, s):
    return np.resize(x, s)

def load_filtering(filter_name, target_size):
    # Replace with your actual implementation
    pass

def rescale(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

def convolutional_barycenter(data, alpha, area_weights, blur_all, entropy_limit, options):
    # Replace with your actual implementation
    pass

def main():
    rep = '../results/shapes/'
    if not os.path.exists(rep):
        os.makedirs(rep)

    # Read data
    target_size = 199
    shape_list = [2, 3, 4]
    n_images = len(shape_list)
    data = np.zeros((target_size * target_size, n_images))

    for i in range(n_images):
        im = io.imread(f'shape{shape_list[i]}filled.png')
        im = color.rgb2gray(im.astype(np.double))

        padding = abs(im.shape[1] - im.shape[0])
        pad1 = padding // 2
        pad2 = padding - pad1

        if im.shape[0] < im.shape[1]:
            im = np.vstack([np.zeros((pad1, im.shape[1])), im, np.zeros((pad2, im.shape[1]))])
        elif im.shape[1] < im.shape[0]:
            im = np.hstack([np.zeros((im.shape[0], pad1)), im, np.zeros((im.shape[0], pad2))])

        im = 1 - image_resize(im, [target_size, target_size])
        im[im < 0] = 0

        im = im > 0.01
        im = im + 1e-3
        im = im / np.sum(im)

        data[:, i] = im.flatten()

    n = data.shape[0]
    area_weights = np.ones(n) / n
    data = data * n

    entropies = -np.sum(data * np.log(data) * area_weights[:, np.newaxis], axis=0)
    max_entropy = np.max(entropies)

    # Set up blur
    filter_size = 1.5
    im_size = [target_size, target_size]

    if hasattr(np, 'imfilter'):
        h = gaussian_filter(np.zeros((1, max(im_size))), filter_size)
        h = h / np.sum(h)
        im_blur = lambda x: gaussian_filter(gaussian_filter(x, h, mode='constant'), h.T, mode='constant')
    else:
        blur = load_filtering('imgaussian', target_size)
        im_blur = lambda x: blur(x, filter_size)

    plt.imshow(im_blur(data[:, 0].reshape(im_size)))
    plt.show()

    blur_column = lambda x: im_blur(x.reshape(im_size)).flatten()
    blur_all2 = lambda x: np.concatenate([blur_column(xi) for xi in np.hsplit(x, x.shape[1])], axis=0)
    blur_all = lambda x: blur_all2(blur_all2(x))

    plt.imshow(blur_all(data[:, 0]).reshape(im_size))
    plt.show()

    # Compute barycenter
    entropy_limit = np.inf

    steps = np.linspace(0, 1, 5)

    averages = [[None] * len(steps) for _ in range(len(steps))]
    barycenters = [[None] * len(steps) for _ in range(len(steps))]

    options = {'niter': 400, 'verb': 2, 'tol': 1e-15}
    resh = lambda x: np.reshape(x, im_size)

    for i, w in enumerate([
        [0, 0, 1],
        [1, 0, 3], [0, 1, 3],
        [1, 0, 1], [1, 1, 2], [0, 1, 1],
        [3, 0, 1], [2, 1, 1], [1, 2, 1], [0, 3, 1],
        [1, 0, 0], [3, 1, 0], [1, 1, 0], [1, 3, 0], [0, 1, 0]
    ]):
        alpha = np.array(w) / np.sum(w)
        barycenters[i] = convolutional_barycenter(data, alpha, area_weights, blur_all, entropy_limit, options)
        u = rescale(-resh(barycenters[i]))
        io.imsave(f'{rep}shape-barycenter-{i + 1}.png', u)
        io.imsave(f'{rep}tresh-barycenter-{i + 1}.png', (u > 0.5).astype(np.double))

if __name__ == "__main__":
    main()

FileNotFoundError: No such file: '/Users/acram/Desktop/VSCode_projects/advanced_ml/Convolutionnal-Wasserstein-Distances/notebooks/augustin/shape2filled.png'